<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto (Canada)</font></h1>


## Introduction

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in Toronto</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>


Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip3 install geocoder
import csv
import time
from pprint import pprint
import geocoder 
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.5
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.5
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Libraries imported.


<a id='item1'></a>


## 1. Download and Explore Dataset


Neighborhood has a total of 6 boroughs and 140 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

The information about neighborhood and boroughs is taken using the wikipedia[https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto] and create a csv file. 

To obtain the geocoordenate of each position there is a python library geocoder, which allows to obtain latitude and logitude by the name of a place.



For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.


In [2]:
df = pd.read_csv('Toronto_Neighborhoods.csv',sep=';', names=['code','neighborhoods','borobought','neighborhood2','latitude','longitude'], delim_whitespace=False)

df.head(140)

,code,neighborhoods,borobought,neighborhood2,latitude,longitude
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,NaN,NaN
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,NaN,NaN
2,20,Alderwood,Etobicoke,Alderwood,NaN,NaN
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,NaN,NaN
4,42,Banbury-Don Mills,North York,Don Mills,NaN,NaN
5,34,Bathurst Manor,North York,Bathurst Manor,NaN,NaN
6,76,Bay Street Corridor,Old City of Toronto,"Bay Street, Financial District",NaN,NaN
7,52,Bayview Village,North York,Bayview Village,NaN,NaN
8,49,Bayview Woods-Steeles,North York,Bayview Woods,NaN,NaN
9,39,Bedford Park-Nortown,North York,"Bedford Park, Ledbury Park, and Nortown",NaN,NaN


In [3]:

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
toronto_data = pd.DataFrame(columns=column_names)

# get location raw data
for i,row in df.iterrows():        
    borobrough = row['borobought']
    neighborhood = row['neighborhoods']    
    latitude = np.nan
    longitude = np.nan      
    res = geocoder.arcgis(f"'{row['neighborhoods']}, Toronto, Canada'")
    if res:
        location = res.json        
        latitude = location['lat']
        longitude = location['lng']
    print(f"{borobrough} - {neighborhood} {latitude} {longitude} ")
    toronto_data = toronto_data.append({'Borough': borobrough, 'Neighborhood': neighborhood, 'Latitude':latitude ,'Longitude':longitude}, ignore_index=True)
    


Scarborough - Agincourt North 43.809300000000064 -79.26706999999999 
Scarborough - Agincourt South-Malvern West 43.787360000000035 -79.26934999999997 
Etobicoke - Alderwood 43.60496000000006 -79.54115999999993 
Old City of Toronto - Annex 43.66936000000004 -79.40279999999996 
North York - Banbury-Don Mills 43.740410000000054 -79.34851999999995 
North York - Bathurst Manor 43.763780000000054 -79.45476999999994 
Old City of Toronto - Bay Street Corridor 43.657710000000066 -79.38617999999997 
North York - Bayview Village 43.777100000000075 -79.37956999999994 
North York - Bayview Woods-Steeles 43.794850000000054 -79.38221999999996 
North York - Bedford Park-Nortown 43.73066000000006 -79.42449999999997 
York - Beechborough-Greenbrook 43.69311000000005 -79.47830999999996 
Scarborough - Bendale 43.75963000000007 -79.25738999999999 
Scarborough - Birchcliffe-Cliffside 43.694720000000075 -79.26459999999997 
North York - Black Creek 43.76636000000008 -79.52149999999995 
Old City of Toronto - Bl

In [4]:
toronto_data.head(140)
toronto_data.to_csv('toronto_neighborhood_position.csv', index = True)

#### Load and explore the data


The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.


In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.read_csv('toronto_neighborhood_position.csv', sep=',', names=column_names, skiprows=1)

Take a look at the empty dataframe to confirm that the columns are as intended.


In [6]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Agincourt North,43.809300,-79.267070
1,Scarborough,Agincourt South-Malvern West,43.787360,-79.269350
2,Etobicoke,Alderwood,43.604960,-79.541160
3,Old City of Toronto,Annex,43.669360,-79.402800
4,North York,Banbury-Don Mills,43.740410,-79.348520
5,North York,Bathurst Manor,43.763780,-79.454770
6,Old City of Toronto,Bay Street Corridor,43.657710,-79.386180
7,North York,Bayview Village,43.777100,-79.379570
8,North York,Bayview Woods-Steeles,43.794850,-79.382220
9,North York,Bedford Park-Nortown,43.730660,-79.424500


Then let's loop through the data and fill the dataframe one row at a time.


Quickly examine the resulting dataframe.


And make sure that the dataset has all 6 boroughs and 140 neighborhoods.


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 6 boroughs and 140 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [8]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.


However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in North York. So let's slice the original dataframe and create a new dataframe of the Manhattan data.


In [10]:
north_york_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
north_york_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Banbury-Don Mills,43.74041,-79.34852
1,North York,Bathurst Manor,43.76378,-79.45477
2,North York,Bayview Village,43.77710,-79.37957
3,North York,Bayview Woods-Steeles,43.79485,-79.38222
4,North York,Bedford Park-Nortown,43.73066,-79.42450


Let's get the geographical coordinates of Manhattan.


In [11]:
address = 'North York, Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


As we did with all of Toronto City, let's visualizat North York the neighborhoods in it.


In [12]:
# create map of Manhattan using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(north_york_data['Latitude'], north_york_data['Longitude'], north_york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [43]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Let's explore the first neighborhood in our dataframe.


Get the neighborhood's name.


In [14]:
north_york_data.loc[0, 'Neighborhood']

'Banbury-Don Mills'

Get the neighborhood's latitude and longitude values.


In [15]:
neighborhood_latitude = north_york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = north_york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_york_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Banbury-Don Mills are 43.74041000000005, -79.34851999999995.


#### Now, let's get the top 100 venues that are in Banbury-Don Mills within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [16]:
# type your answer here
radius = 500
search_query ='*'
latitude = neighborhood_latitude
longitude = neighborhood_longitude
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=T2YAX2CSK0IOORW1JKGZLVT2B20MIPTKB15AED0RJASVHZI2&client_secret=UPCIL51R1ZEZAIBU02RODEVFWK3FQ3G3RAFHB20W2TCX0DI2&ll=43.74041000000005,-79.34851999999995&v=20180605&query=*&radius=500&limit=100'

Double-click **here** for the solution.

<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 


Send the GET request and examine the resutls


In [17]:
results = requests.get(url).json()
#results['response']
results['meta']['code'] == 200

True

In [18]:
with open('foursquare_respones.json', 'w') as my_data_file:
    json.dump(results, my_data_file)

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-20-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Baretto Caffé,Café,43.744456,-79.346460
1,Shoppers Drug Mart,Pharmacy,43.737261,-79.345868
2,LCBO,Liquor Store,43.736323,-79.346115
3,Tim Hortons,Coffee Shop,43.737491,-79.343881
4,Top Queen nails,Spa,43.736158,-79.347241


And how many venues were returned by Foursquare?


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


<a id='item2'></a>


## 2. Explore Neighborhoods in North York


#### Let's create a function to repeat the same process to all the neighborhoods in North York


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results0 = requests.get(url).json()
        if results0['meta']['code'] != 200:
            print('Error in {}'.format(name))
            continue
        
        results = results0["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _north_york_venues_.


In [23]:
# type your answer here
north_york_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Banbury-Don Mills,43.74041,-79.34852
1,North York,Bathurst Manor,43.76378,-79.45477
2,North York,Bayview Village,43.77710,-79.37957
3,North York,Bayview Woods-Steeles,43.79485,-79.38222
4,North York,Bedford Park-Nortown,43.73066,-79.42450


In [24]:
north_york_data['Neighborhood']

north_york_venues = getNearbyVenues(north_york_data['Neighborhood'], north_york_data['Latitude'], north_york_data['Longitude'])

Banbury-Don Mills
Bathurst Manor
Bayview Village
Bayview Woods-Steeles
Bedford Park-Nortown
Black Creek
Bridle Path-Sunnybrook-York Mills
Brookhaven-Amesbury
Clanton Park
Don Valley Village
Downsview-Roding-CFB
Englemount-Lawrence
Flemingdon Park
Glenfield-Jane Heights
Henry Farm
Hillcrest Village
Humber Summit
Humbermede
Lansing-Westgate
Maple Leaf
Newtonbrook East
Newtonbrook West
Parkwoods-Donalda
Pelmo Park-Humberlea
Pleasant View
Rustic
St. Andrew-Windfields
Victoria Village
Westminster-Branson
Willowdale East
Willowdale West
York University Heights
Yorkdale-Glen Park


Double-click **here** for the solution.

<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 


#### Let's check the size of the resulting dataframe


In [25]:
print(north_york_venues.shape)
north_york_venues.head()

(161, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Banbury-Don Mills,43.74041,-79.34852,Baretto Caffé,43.744456,-79.346460,Café
1,Banbury-Don Mills,43.74041,-79.34852,Shoppers Drug Mart,43.737261,-79.345868,Pharmacy
2,Banbury-Don Mills,43.74041,-79.34852,LCBO,43.736323,-79.346115,Liquor Store
3,Banbury-Don Mills,43.74041,-79.34852,Tim Hortons,43.737491,-79.343881,Coffee Shop
4,Banbury-Don Mills,43.74041,-79.34852,Top Queen nails,43.736158,-79.347241,Spa


Let's check how many venues were returned for each neighborhood


In [26]:
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Banbury-Don Mills,5,5,5,5,5,5
Bathurst Manor,5,5,5,5,5,5
Bayview Village,4,4,4,4,4,4
Bayview Woods-Steeles,2,2,2,2,2,2
Bedford Park-Nortown,8,8,8,8,8,8
Black Creek,9,9,9,9,9,9
Bridle Path-Sunnybrook-York Mills,7,7,7,7,7,7
Clanton Park,2,2,2,2,2,2
Don Valley Village,13,13,13,13,13,13


#### Let's find out how many unique categories can be curated from all the returned venues


In [27]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 81 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [28]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + list(north_york_onehot.columns[:-1])
manhattan_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()

,Accessories Store,African Restaurant,Arcade,Art Gallery,Astrologer,Athletics & Sports,Bakery,Bank,Baseball Field,Basketball Court,Beer Store,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Bridal Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym,College Quad,Construction & Landscaping,Convenience Store,Cosmetics Shop,Discount Store,Dog Run,Falafel Restaurant,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Driving Range,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Home Service,Hotel,IT Services,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lawyer,Liquor Store,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Science Museum,Seafood Restaurant,Shopping Mall,Skating Rink,Snack Place,Spa,Tea Room,Thrift / Vintage Store,Trail,Wings Joint,Women's Store,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Banbury-Don Mills
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Banbury-Don Mills
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Banbury-Don Mills
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Banbury-Don Mills
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,Banbury-Don Mills


And let's examine the new dataframe size.


In [29]:
north_york_onehot.shape

(161, 82)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [30]:
north_york_grouped = north_york_onehot.groupby('Neighborhood').mean().reset_index()
north_york_grouped

,Neighborhood,Accessories Store,African Restaurant,Arcade,Art Gallery,Astrologer,Athletics & Sports,Bakery,Bank,Baseball Field,Basketball Court,Beer Store,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Bridal Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym,College Quad,Construction & Landscaping,Convenience Store,Cosmetics Shop,Discount Store,Dog Run,Falafel Restaurant,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,Golf Driving Range,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Home Service,Hotel,IT Services,Ice Cream Shop,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lawyer,Liquor Store,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Playground,Plaza,Pool,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Science Museum,Seafood Restaurant,Shopping Mall,Skating Rink,Snack Place,Spa,Tea Room,Thrift / Vintage Store,Trail,Wings Joint,Women's Store
0,Banbury-Don Mills,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.0,0.200,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.2,0.00,0.000000,0.000000,0.0,0.00
1,Bathurst Manor,0.0,0.0,0.2,0.0,0.00,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.200000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.0,0.000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.2,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.00
2,Bayview Village,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.0,0.000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.250000,0.0,0.00
3,Bayview Woods-Steeles,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.5,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000,0.0,0.0,0.000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.500000,0.000000,0.000000,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.00
4,Bedford Park-Nortown,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.125000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.125,0.0,0.0,0.125,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.125000,0.000000,0.000000,0.0,0.00,0.00,0.0,0.000000,0.250000,0.000000,

#### Let's confirm the new size


In [31]:
north_york_grouped.shape

(31, 82)

#### Let's print each neighborhood along with the top 5 most common venues


In [32]:
num_top_venues = 5

for hood in north_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_york_grouped[north_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Banbury-Don Mills----
          venue  freq
0      Pharmacy   0.2
1   Coffee Shop   0.2
2           Spa   0.2
3  Liquor Store   0.2
4          Café   0.2


----Bathurst Manor----
               venue  freq
0               Park   0.2
1             Arcade   0.2
2         Playground   0.2
3     Baseball Field   0.2
4  Convenience Store   0.2


----Bayview Village----
                        venue  freq
0                       Trail  0.25
1          Golf Driving Range  0.25
2                 Flower Shop  0.25
3  Construction & Landscaping  0.25
4           Accessories Store  0.00


----Bayview Woods-Steeles----
               venue  freq
0               Park   0.5
1            Dog Run   0.5
2          Juice Bar   0.0
3      Moving Target   0.0
4  Mobile Phone Shop   0.0


----Bedford Park-Nortown----
            venue  freq
0  Sandwich Place  0.25
1   Grocery Store  0.12
2    Skating Rink  0.12
3       Juice Bar  0.12
4    Liquor Store  0.12


----Black Creek----
                venue 

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_york_grouped['Neighborhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Banbury-Don Mills,Café,Pharmacy,Spa,Liquor Store,Coffee Shop,Women's Store,College Quad,Butcher,Caribbean Restaurant,Chinese Restaurant
1,Bathurst Manor,Convenience Store,Arcade,Park,Playground,Baseball Field,Construction & Landscaping,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop
2,Bayview Village,Golf Driving Range,Construction & Landscaping,Trail,Flower Shop,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym
3,Bayview Woods-Steeles,Park,Dog Run,Women's Store,Construction & Landscaping,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym
4,Bedford Park-Nortown,Sandwich Place,Skating Rink,Liquor Store,Park,Juice Bar,Coffee Shop,Grocery Store,Discount Store,Cosmetics Shop,Convenience Store


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [35]:
# set number of clusters
kclusters = 5

north_york_grouped_clustering = north_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_york_merged = north_york_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

north_york_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Banbury-Don Mills,43.74041,-79.34852,2.0,Café,Pharmacy,Spa,Liquor Store,Coffee Shop,Women's Store,College Quad,Butcher,Caribbean Restaurant,Chinese Restaurant
1,North York,Bathurst Manor,43.76378,-79.45477,2.0,Convenience Store,Arcade,Park,Playground,Baseball Field,Construction & Landscaping,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop
2,North York,Bayview Village,43.77710,-79.37957,2.0,Golf Driving Range,Construction & Landscaping,Trail,Flower Shop,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym
3,North York,Bayview Woods-Steeles,43.79485,-79.38222,1.0,Park,Dog Run,Women's Store,Construction & Landscaping,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym
4,North York,Bedford Park-Nortown,43.73066,-79.42450,2.0,Sandwich Place,Skating Rink,Liquor Store,Park,Juice Bar,Coffee Shop,Grocery Store,Discount Store,Cosmetics Shop,Convenience Store


Finally, let's visualize the resulting clusters


In [37]:
import math
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighborhood'], north_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if math.isnan(cluster):
        continue
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters



<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [38]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 0, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Humber Summit,Construction & Landscaping,Furniture / Home Store,Women's Store,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym,College Quad
23,Pelmo Park-Humberlea,Furniture / Home Store,Skating Rink,Moving Target,Flower Shop,Fast Food Restaurant,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant
31,York University Heights,Furniture / Home Store,Big Box Store,Gas Station,Miscellaneous Shop,Fast Food Restaurant,Pizza Place,Discount Store,Café,Caribbean Restaurant,Chinese Restaurant


#### Cluster 2


In [39]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 1, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Bayview Woods-Steeles,Park,Dog Run,Women's Store,Construction & Landscaping,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym
15,Hillcrest Village,Park,Residential Building (Apartment / Condo),Women's Store,College Quad,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym
18,Lansing-Westgate,Park,Health & Beauty Service,Women's Store,Construction & Landscaping,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym
30,Willowdale West,Park,Mobile Phone Shop,Women's Store,College Quad,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym


#### Cluster 3


In [40]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 2, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Banbury-Don Mills,Café,Pharmacy,Spa,Liquor Store,Coffee Shop,Women's Store,College Quad,Butcher,Caribbean Restaurant,Chinese Restaurant
1,Bathurst Manor,Convenience Store,Arcade,Park,Playground,Baseball Field,Construction & Landscaping,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop
2,Bayview Village,Golf Driving Range,Construction & Landscaping,Trail,Flower Shop,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym
4,Bedford Park-Nortown,Sandwich Place,Skating Rink,Liquor Store,Park,Juice Bar,Coffee Shop,Grocery Store,Discount Store,Cosmetics Shop,Convenience Store
5,Black Creek,Grocery Store,Skating Rink,Chinese Restaurant,Pharmacy,Basketball Court,Athletics & Sports,Shopping Mall,Gas Station,Snack Place,Convenience Store
6,Bridle Path-Sunnybrook-York Mills,Trail,Café,Bus Line,Coffee Shop,College Gym,Bookstore,College Quad,Women's Store,Construction & Landscaping,Caribbean Restaurant
8,Clanton Park,Home Service,IT Services,Women's Store,Convenience Store,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym,College Quad
9,Don Valley Village,Bakery,Grocery Store,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Caribbean Restaurant,Park,Pizza Place,Beer Store
11,Englemount-Lawrence,Accessories Store,Park,Art Gallery,Lawyer,Bank,Construction & Landscaping,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop
12,Flemingdon Park,Japanese Restaurant,Grocery Store,Sandwich Place,Falafel Restaurant,Gym,Gym / Fitness Center,Coffee Shop,Middle Eastern Restaurant,Pakistani Restaurant,Pharmacy


#### Cluster 4


In [41]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 3, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downsview-Roding-CFB,Baseball Field,Business Service,Women's Store,Convenience Store,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym,College Quad


#### Cluster 5


In [42]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 4, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Newtonbrook West,Park,Women's Store,Construction & Landscaping,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym,College Quad
26,St. Andrew-Windfields,Park,Women's Store,Construction & Landscaping,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym,College Quad
27,Victoria Village,Park,Women's Store,Construction & Landscaping,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Gym,College Quad


<hr>

Copyright © 2018 [Cognitive Class](https://cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
